In [10]:
import pandas as pd
import numpy as np

## Ładowanie i przetwarzanie danych wejściowych

In [11]:
# Funkcja służąca do wczytania danych z wybranego pliku
def load_data_from_file(file_name):
    # Ładujemy nasze dane z repozytorium na GitHub
    data = pd.read_csv('https://raw.githubusercontent.com/nerooc/device-downtime-detection/main/data/{0}'.format(file_name), header=None).to_numpy()
    
    # Nie potrzebujemy indeksów 1, 2, 3, 4... itp., więc usuwamy pierwszą kolumnę
    COLUMNWISE = 1
    data = np.delete(data, obj=0, axis=COLUMNWISE)

    return data

# Funkcja służąca do wczytania wszystkich danych
def load_all_data(num_of_iters):
  # W tej liście przechowywać będziemy wszystkie dane z urządzenia
  data_blocks = []

  # Manipulacja danymi, żeby w każdej iteracji wczytać dane zarówno z pliku normal
  # jak i z pliku fault
  for i in range(num_of_iters):
    file_index = i + 1
    file_name_normal = f'data{file_index}_normal.csv'
    file_name_fault = f'data{file_index}_fault.csv'

    # Dodatkowe wskazanie, że to odczyt dla działającego urządzenia (1)
    normal_data_block = [load_data_from_file(file_name_normal), 1]
    # Dodatkowe wskazanie, że to odczyt dla niedziałającego urządzenia (0)
    fault_data_block = [load_data_from_file(file_name_fault), 0]

    # Dodajemy te pakiety odczytów w każdej iteracji do wszystkich odczytów
    data_blocks.append(normal_data_block)
    data_blocks.append(fault_data_block)

  return data_blocks

## Normalizacja i podział danych

In [12]:
# Funkcja normalizująca dataframe'y
def normalize_data(df):
  for column in df:
    df[column] = df[column] / df[column].max()
  return df

# Funkcja dzieląca dane na znormalizowane bloki działające i awaryjne
def split_working_faulty_blocks(normalized=False):
  # Wczytujemy dane
  # Jest 54 dla fault i 53 dla normal, ale wczytamy jedno mniej dla symetrii
  data_blocks = load_all_data(num_of_iters=53)
  # Dodajmy ostatni blok danych
  data_fault_last = pd.read_csv('https://raw.githubusercontent.com/nerooc/device-downtime-detection/main/data/data54_fault.csv', header=None).to_numpy()
  # Nie potrzebujemy indeksów 1, 2, 3, 4... itp., więc usuwamy pierwszą kolumnę
  COLUMNWISE = 1
  data_fault_last = np.delete(data_fault_last, obj=0, axis=COLUMNWISE)
  data_blocks.append([data_fault_last, 0])

  # Bloki działających urządzeń
  working_blocks = [block[0] for block in data_blocks if block[1] == 1]

  # Bloki niedziałających urządzeń
  faulty_blocks = [block[0] for block in data_blocks if block[1] == 0]

  working_blocks_np = working_blocks[0].copy()
  faulty_blocks_np = faulty_blocks[0].copy()

  for i in range(len(working_blocks) - 1):
    real_index = i + 1
    working_blocks_np = np.concatenate((working_blocks_np, working_blocks[real_index]), axis=0)

  for i in range(len(faulty_blocks) - 1):
    real_index = i + 1
    faulty_blocks_np = np.concatenate((faulty_blocks_np, faulty_blocks[real_index]), axis=0)

  working_blocks_df = pd.DataFrame(working_blocks_np)
  faulty_blocks_df = pd.DataFrame(faulty_blocks_np)
 
  if normalized:
    return (normalize_data(working_blocks_df), normalize_data(faulty_blocks_df))

  return (working_blocks_df,faulty_blocks_df)


In [13]:
# przemieszajmy bloki danych
from sklearn.utils import shuffle

NUMBER_OF_TIMESTAMPS = 8

def get_input_list_completed():

  (working_blocks_df, faulty_blocks_df) = split_working_faulty_blocks(normalized=True)

  # utworzmy pakiety danych po NUMBER_OF_TIMESTAMPS chwil czasowych nalezy miec na uwadze
  # ze dane moga nie byc podzielne akurat przez NUMBER_OF_TIMESTAMPS wiec usunmy kilka chwil czasowych
  # na koncu obu zbiorow (zapisow dla dzialajacego i niedzialajaceg urzadzenia)
  working_blocks_rest = len(working_blocks_df) % NUMBER_OF_TIMESTAMPS
  faulty_blocks_rest = len(faulty_blocks_df) % NUMBER_OF_TIMESTAMPS

  working_blocks_df = working_blocks_df[:-working_blocks_rest]
  faulty_blocks_df = faulty_blocks_df[:-faulty_blocks_rest]

  working_blocks_packets_df = np.array_split(working_blocks_df, int(len(working_blocks_df) / NUMBER_OF_TIMESTAMPS))
  faulty_blocks_packets_df = np.array_split(faulty_blocks_df, int(len(faulty_blocks_df) / NUMBER_OF_TIMESTAMPS))

  # dodajmy klasy zeby bylo wiadomo co jest czym -> nim przemieszamy pomiedzy klasami
  working_blocks_packets_with_class = []

  for i in range(len(working_blocks_packets_df)):
    working_blocks_packets_with_class.append( [working_blocks_packets_df[i].to_numpy(), 1] )

  faulty_blocks_packets_with_class = []

  for i in range(len(faulty_blocks_packets_df)):
    faulty_blocks_packets_with_class.append( [faulty_blocks_packets_df[i].to_numpy(), 0] )

  input_list_completed = working_blocks_packets_with_class.copy()
  input_list_completed.extend(faulty_blocks_packets_with_class)

  # przemieszajmy dane po zlaczeniu
  input_list_completed = shuffle(input_list_completed)

  return input_list_completed

In [14]:
def prepare_input(input_list_completed):
  # przemieszajmy dane po zlaczeniu
  X = []
  y = []

  for i in range(len(input_list_completed)):
    X.append(input_list_completed[i][0])
    y.append(input_list_completed[i][1])

  return (X, y)

## Test działania metod

In [8]:
# print(split_working_faulty_blocks())